In [ ]:
!pip install -r requirements.txt
!pip install tensorflow
!pip install lifelines
# Install Mol2vec
!pip install git+https://github.com/samoturk/mol2vec
!pip install pyfaidx

In [ ]:
# all needed packages
import pandas as pd
from copy import deepcopy
import math as math
import numpy as np
from math import exp
# import argparse
import json,csv, pickle
import itertools,collections
import matplotlib.pyplot as plt

# ML packages
from sklearn.metrics import *
from sklearn.metrics import mean_squared_error, r2_score
from lifelines.utils import concordance_index
from sklearn.preprocessing import StandardScaler
# from joblib import Parallel, delayed
# import multiprocessing
import xgboost as xgb

# normalization packages
from sklearn.preprocessing import MinMaxScaler, minmax_scale
import scipy
from scipy import *

# DL Keras
from keras.wrappers.scikit_learn import KerasRegressor
from keras.preprocessing.sequence import skipgrams
from keras.layers import Embedding, Input, Reshape, Dense, concatenate
from keras.models import Sequential, Model

# for protein embedding
import biovec

# for drugs embedding
# import tensorflow as tf
# from tensorflow import keras
# import deepchem as dc
# from deepchem.feat import Featurizer
# from deepchem.models.optimizers import ExponentialDecay, Adam
# from deepchem.models.seqtoseq import AspuruGuzikAutoEncoder
# from deepchem.metrics import to_one_hot
# from deepchem.models.graph_models import GraphConvModel,L2Loss,Dense,Reshape,Dropout 

# Import my files
from training_functions import *
from pathScores_functions import *
from evaluation import *

In [ ]:
#load data
all_DTA_info = pd.read_csv('/DAVIS.csv')

In [ ]:
# import trained model
pv = biovec.models.load_protvec('/home/amit/BindingAffinity/Trained_models/swissprot-reviewed-protvec.model')

Embed = {}
seqEmbed = []
sumEmbed = []
avgEmbed = []
allEmbed = []

all_DT_df = pd.read_csv("/DAVIS.csv")
all_DT_df['Seq'] = all_DT_df['Seq'].astype(str)
all_DT_df = all_DT_df.dropna()

# Remove rows with missing values in the 'Seq' column
all_DT_df = all_DT_df.dropna(subset=['Seq'])
all_DT_df.dropna(inplace=True)


for i in range(all_DT_df.shape[0]):
    embedding = pv.to_vecs(all_DT_df['Seq'][i])
    # Dictionary to get embedding by target ID
    Embed[all_DT_df['pdbID'][i]] = embedding
    
    allem = list(itertools.chain(embedding[0],embedding[1],embedding[2]))
    #concatenate 3d
    allEmbed.append(allem)
    #list of embedding 3D
    seqEmbed.append(embedding)
    
    # 1D embedding by taking sum of 3 ists
    sumE = embedding[0]+embedding[1]+embedding[2]
    sumEmbed.append(sumE)
    
    #1D embedding by taking the average
    #avgEmbed.append(sumE/3)
    
#avgEmbed = np.array(avgEmbed)
sumEmbed = np.array(sumEmbed)
sumEmbed.shape

In [ ]:
dr_pdb_Sm = list(all_DT_df['SMILES'])
df = pd.DataFrame({'smiles': dr_pdb_Sm})

df = df.dropna(subset=['smiles'])
df['mol'] = df['smiles'].apply(lambda x: Chem.MolFromSmiles(x))

df['sentence'] = df.apply(lambda x: MolSentence(mol2alt_sentence(x['mol'], 1)), axis=1)

In [ ]:
# Check whether the dataframe has valid smiles or not

def validate_smiles(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        return True
    except:
        return False

df['valid_smiles'] = df['smiles'].apply(validate_smiles)

for i, row in df.iterrows():
    print(row['smiles'], row['valid_smiles'])

In [ ]:
# Import Mol2Vec pretrained model
model = word2vec.Word2Vec.load('Trained_models/model_300dim.pkl')
df['mol2vec'] = [DfVec(x) for x in sentences2vec(df['sentence'], model, unseen='UNK')]
pdb_sm_EMBED = np.array([x.vec for x in df['mol2vec']])

In [ ]:
np.save('/data/amit/Mol2Vec_EMB_BindingDB_excluding_bad_conformers.npy', pdb_sm_EMBED)
pdb_sm_EMBED = np.load('/data/amit/Mol2Vec_EMB_BindingDB_excluding_bad_conformers.npy',allow_pickle=True)
pdb_sm_unique = np.array(list(ligand_un_df['SMILES']))

In [ ]:
df1 = pd.DataFrame({'smiles': pdb_sm_unique})
df1 = df1.dropna(subset=['smiles'])
#Transforming SMILES to MOL
df1['mol'] = df1['smiles'].apply(lambda x: Chem.MolFromSmiles(x))

#Constructing sentences
df1['sentence'] = df1.apply(lambda x: MolSentence(mol2alt_sentence(x['mol'], 1)), axis=1)

#Extracting embeddings to a numpy.array
df1['mol2vec'] = [DfVec(x) for x in sentences2vec(df1['sentence'], model, unseen='UNK')]
pdb_sm_EMBED_u = np.array([x.vec for x in df1['mol2vec']])


In [ ]:
# Write Sequence Embeddings and SMILES Embeddings into file
Pr_EMBED_df = pd.DataFrame.from_dict(sumEmbed)
Pr_EMBED_df.to_csv('/EMBED/Pr_ProtVec_EMBED.txt',sep=' ',index=None,header=None)

pdb_EMBED_df = pd.DataFrame.from_dict(pdb_sm_EMBED)
pdb_EMBED_df.to_csv('/EMBED/Dr_Mol2Vec_EMBED.txt',sep=' ',index=None,header=None)